## Environmental Intensity Level Regressions and Dickey–Fuller test

In the notebook 'PredictingTimeSeries_&_PilotStock_CompDescription', we ran several linear regressions combining different features and we saw it yield a high R2 score. In this notebook, we will run the regressions but focus on their coefficients. 

Also, we are going to do some Dickey-Fuller test

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import warnings
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import requests
from bs4 import BeautifulSoup
import os
import statsmodels.api as sm
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df=pd.read_csv('/Users/maralinetorres/Documents/GitHub/Predicting-Environmental-and-Social-Actions/Datasets/Environmental_impact_cleaned.csv')
df.head()

,ISIN,Year,CompanyName,Country,Industry(Exiobase),EnvironmentalIntensity(Sales),EnvironmentalIntensity(OpInc),TotalEnvironmentalCost,WorkingCapacity,FishProductionCapacity,...,SDG14.c,SDG15.1,SDG15.2,SDG15.5,%Imputed,Env_intensity,industry_avg,industry_avg_year,Industry_indicator_year,Environmental_Growth
0,DE0005545503,2016,1&1 DRILLISCH AG,Germany,Post and telecommunications (64),-0.07%,-0.82%,"-539,318","-525,027",-169,...,-6,67,67,-22,23%,-0.0007,-0.020506,-0.020740,1,NaN
1,GB00B1YW4409,2010,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.12%,-0.11%,"-1,055,812","-1,032,103",-277,...,-4,51,51,-43,10%,-0.0012,-0.028537,-0.006402,1,NaN
2,GB00B1YW4409,2011,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.16%,-0.16%,"-961,875","-940,402",-246,...,-3,38,38,-39,9%,-0.0016,-0.028537,-0.009838,1,33.333333
3,GB00B1YW4409,2012,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.15%,NaN,"-722,999","-706,893",-183,...,-2,27,27,-30,8%,-0.0015,-0.028537,-0.024437,1,-6.250000
4,US88579Y1010,2010,3M COMPANY,United States,Activities of membership organisation n.e.c. (91),-7.90%,-35.45%,"-2,105,919,763","-1,924,672,080","-439,506",...,-423,"3,772","3,772","-79,722",1%,-0.0790,-0.175838,-0.084583,1,NaN


In [ ]:
years = [2016, 2017, 2018]
df_industry = df.groupby('Industry(Exiobase)').count()['CompanyName'].reset_index()
industries = df_industry[df_industry['CompanyName'] > 3]['Industry(Exiobase)']
df_industry_count4 = df[df['Industry(Exiobase)'].isin(industries)]
df_c = df_industry_count4.copy()
def predictiveModel(outcomeYear, pastYears, df_c):
    years.sort()
    for year in years:
        data = df_c[df_c['Year'] == year]
        data = data.loc[:,['CompanyName','Env_intensity','industry_avg_year']]
        data.rename(columns={'Env_intensity': f'Env_intensity_{year}','industry_avg_year':f'industry_avg_year_{year}'}, inplace=True) 
        if(year == min(years)):
            data1 = pd.DataFrame(data)
        else:
            data2 = pd.merge(data1, data, on=["CompanyName"])
            data1 = data2.copy()
    data3 = df_c[df_c['Year'] == outcomeYear]
    data3 = data3[['CompanyName','Env_intensity','industry_avg_year']]
    data3.rename(columns={'Env_intensity': f'Env_intensity_{outcomeYear}','industry_avg_year':f'industry_avg_year_{outcomeYear}'}, inplace=True) 
    data3 = pd.merge(data3, data2, on=["CompanyName"])
    
    filter_col = [col for col in data3 if ((col.startswith('Env_intensity') and not(col.endswith(f'{outcomeYear}')))) or ((col.startswith('industry_avg_year') and not(col.endswith(f'{outcomeYear}'))))]
    outcome_col = [col for col in data3 if (col.startswith('Env_intensity') and col.endswith(f'{outcomeYear}'))]
    X=data3[filter_col]
    y=data3[outcome_col]
    
    x = sm.add_constant(X)
    print(sm.OLS(y, x).fit().summary())

In [ ]:
predictiveModel(2019, years, df_c)

                            OLS Regression Results                            
Dep. Variable:     Env_intensity_2019   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     1340.
Date:                Thu, 15 Jul 2021   Prob (F-statistic):               0.00
Time:                        22:30:25   Log-Likelihood:                 1179.1
No. Observations:                1065   AIC:                            -2344.
Df Residuals:                    1058   BIC:                            -2309.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0

In [ ]:
def predictiveModel_v2(outcomeYear, pastYears, df_c):
    years.sort()
    for year in years:
        data = df_c[df_c['Year'] == year]
        data = data.loc[:,['CompanyName','Env_intensity','industry_avg_year','Industry_indicator_year']]
        data.rename(columns={'Env_intensity': f'Env_intensity_{year}','industry_avg_year':f'industry_avg_year_{year}', 'Industry_indicator_year' : f'Industry_indicator_year_{year}'}, inplace=True) 
        if(year == min(years)):
            data1 = pd.DataFrame(data)
        else:
            data2 = pd.merge(data1, data, on=["CompanyName"])
            data1 = data2.copy()
    data3 = df_c[df_c['Year'] == outcomeYear]
    data3 = data3[['CompanyName','Env_intensity']]
    data3.rename(columns={'Env_intensity': f'Env_intensity_{outcomeYear}'}, inplace=True) 
    data3 = pd.merge(data3, data2, on=["CompanyName"])
    
    filter_col = [col for col in data3 if ((col.startswith('Env_intensity') and not(col.endswith(f'{outcomeYear}')))) or ((col.startswith('industry_avg_year') and not(col.endswith(f'{outcomeYear}')))) or ((col.startswith('Industry_indicator_year_') and not(col.endswith(f'{outcomeYear}'))))]            
    outcome_col = [col for col in data3 if (col.startswith('Env_intensity') and col.endswith(f'{outcomeYear}'))]
    X=data3[filter_col]
    y=data3[outcome_col]
    
    x = sm.add_constant(X)
    print(sm.OLS(y, x).fit().summary())

In [ ]:
predictiveModel_v2(2019, years, df_c)

                            OLS Regression Results                            
Dep. Variable:     Env_intensity_2019   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     893.7
Date:                Thu, 15 Jul 2021   Prob (F-statistic):               0.00
Time:                        22:31:46   Log-Likelihood:                 1180.6
No. Observations:                1065   AIC:                            -2341.
Df Residuals:                    1055   BIC:                            -2291.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [ ]:
def predictiveModel_v3(outcomeYear, pastYears, df_c):
    years.sort()
    for year in years:
        data = df_c[df_c['Year'] == year]
        data = data.loc[:,['CompanyName','Env_intensity','industry_avg_year','Industry_indicator_year','Environmental_Growth']]
        data.rename(columns={'Env_intensity': f'Env_intensity_{year}','industry_avg_year':f'industry_avg_year_{year}', 'Industry_indicator_year' : f'Industry_indicator_year_{year}', 'Environmental_Growth': f'Environmental_Growth_{year}'}, inplace=True) 
        if(year == min(years)):
            data1 = pd.DataFrame(data)
        else:
            data2 = pd.merge(data1, data, on=["CompanyName"])
            data1 = data2.copy()
    data2.dropna(inplace=True)
    data3 = df_c[df_c['Year'] == outcomeYear]
    data3 = data3[['CompanyName','Env_intensity']]
    data3.rename(columns={'Env_intensity': f'Env_intensity_{outcomeYear}'}, inplace=True) 
    data3 = pd.merge(data3, data2, on=["CompanyName"])
    
    filter_col = [col for col in data3 if ((col.startswith('Env_intensity') and not(col.endswith(f'{outcomeYear}')))) or ((col.startswith('industry_avg_year') and not(col.endswith(f'{outcomeYear}')))) or ((col.startswith('Industry_indicator_year_') and not(col.endswith(f'{outcomeYear}')))) or ((col.startswith('Environmental_Growth_') and not(col.endswith(f'{outcomeYear}'))))]            
    outcome_col = [col for col in data3 if (col.startswith('Env_intensity') and col.endswith(f'{outcomeYear}'))]
    X=data3[filter_col]
    y=data3[outcome_col]
    
    x = sm.add_constant(X)
    print(sm.OLS(y, x).fit().summary())

In [ ]:
predictiveModel_v3(2019, years, df_c)

                            OLS Regression Results                            
Dep. Variable:     Env_intensity_2019   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     596.2
Date:                Thu, 15 Jul 2021   Prob (F-statistic):               0.00
Time:                        22:36:03   Log-Likelihood:                 1109.2
No. Observations:                1010   AIC:                            -2192.
Df Residuals:                     997   BIC:                            -2128.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

Summary: Using environmental intensity and industry average year have highest explanatory power. The coefficients for industry average of 2016 and 2018 and environmental intensity for 2018 are statistically significant. 2018 EI has highest(1.02) fixed effect on 2019 EI.

## Dickey–Fuller test

In [27]:
from statsmodels.tsa.stattools import adfuller
df=pd.read_csv('Environmental_impact_cleaned.csv')
ind = df.copy()
y2018=list(ind[ind['Year'] == 2018]['Env_intensity'])
y2018=pd.Series(y2018)
X = y2018.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))


mean1=-0.129262, mean2=-0.101468
variance1=0.094017, variance2=0.052920
ADF Statistic: -24.980342
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [ ]:
y2017=list(ind[ind['Year'] == 2017]['Env_intensity'])
y2017=pd.Series(y2017)
from statsmodels.tsa.stattools import adfuller
X = y2017.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.123985, mean2=-0.108138
variance1=0.089221, variance2=0.063000
ADF Statistic: -26.071015
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [ ]:
y2016=list(ind[ind['Year'] == 2016]['Env_intensity'])
y2016=pd.Series(y2016)
from statsmodels.tsa.stattools import adfuller
X = y2016.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -38.998411
p-value: 0.000000
Critical Values:
mean1=-0.128953, mean2=-0.113759
variance1=0.084427, variance2=0.066553
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [ ]:
y2015=list(ind[ind['Year'] == 2015]['Env_intensity'])
y2015=pd.Series(y2015)
from statsmodels.tsa.stattools import adfuller
X = y2015.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.128073, mean2=-0.112539
variance1=0.072975, variance2=0.066954
ADF Statistic: -25.147693
p-value: 0.000000
Critical Values:
	1%: -3.435
	5%: -2.863
	10%: -2.568


In [ ]:
y2014=list(ind[ind['Year'] == 2014]['Env_intensity'])
y2014=pd.Series(y2014)
from statsmodels.tsa.stattools import adfuller
X = y2014.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.113915, mean2=-0.100781
variance1=0.062142, variance2=0.055520
ADF Statistic: -15.509563
p-value: 0.000000
Critical Values:
	1%: -3.435
	5%: -2.864
	10%: -2.568


Summary: The environmental intensity from 2014 to 2018 for each year the data is stational

Next,let's exam the industry average for each year to see whether they are stational

In [ ]:
ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14515 entries, 0 to 14514
Data columns (total 39 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ISIN                                                    14515 non-null  object 
 1   Year                                                    14515 non-null  int64  
 2   CompanyName                                             14515 non-null  object 
 3   Country                                                 14515 non-null  object 
 4   Industry(Exiobase)                                      14515 non-null  object 
 5   EnvironmentalIntensity(Sales)                           14515 non-null  object 
 6   EnvironmentalIntensity(OpInc)                           13700 non-null  object 
 7   TotalEnvironmentalCost                                  14515 non-null  object 
 8   WorkingCapacity                     

Test whether the industry average for each year we used is stationary :


In [ ]:
y2018=list(ind[ind['Year'] == 2018]['industry_avg_year'])
y2018=pd.Series(y2018)
from statsmodels.tsa.stattools import adfuller
X = y2018.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.121867, mean2=-0.108862
variance1=0.032617, variance2=0.026903
ADF Statistic: -10.463661
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [ ]:
y2017=list(ind[ind['Year'] == 2017]['industry_avg_year'])
y2017=pd.Series(y2017)
from statsmodels.tsa.stattools import adfuller
X = y2017.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.117744, mean2=-0.114378
variance1=0.030029, variance2=0.028428
ADF Statistic: -27.227921
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [ ]:
y2016=list(ind[ind['Year'] == 2016]['industry_avg_year'])
y2016=pd.Series(y2016)
from statsmodels.tsa.stattools import adfuller
X = y2016.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.125713, mean2=-0.117000
variance1=0.035538, variance2=0.033986
ADF Statistic: -38.943780
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [ ]:
y2015=list(ind[ind['Year'] == 2015]['industry_avg_year'])
y2015=pd.Series(y2015)
from statsmodels.tsa.stattools import adfuller
X = y2015.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.125579, mean2=-0.115032
variance1=0.038191, variance2=0.033880
ADF Statistic: -20.655671
p-value: 0.000000
Critical Values:
	1%: -3.435
	5%: -2.863
	10%: -2.568


Summary:Each year from 2015 to 2018, the industry average is stational. And the model we used which includes the the industry average is predictable.

Let's take a look at environmental growth

In [28]:
ind['Environmental_Growth']=ind['Environmental_Growth'].fillna(ind['Environmental_Growth'].mean())

In [29]:
y2016=list(ind[ind['Year'] == 2016]['Environmental_Growth'])
y2016=pd.Series(y2016)
from statsmodels.tsa.stattools import adfuller
X = y2016.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=5.411575, mean2=4.908355
variance1=1923.017251, variance2=5805.189898
ADF Statistic: -13.302222
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [30]:
y2017=list(ind[ind['Year'] == 2017]['Environmental_Growth'])
y2017=pd.Series(y2017)
from statsmodels.tsa.stattools import adfuller
X = y2017.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=99.633708, mean2=-5.132693
variance1=5728066.542782, variance2=1756.053671
ADF Statistic: -41.477166
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


In [31]:
y2018=list(ind[ind['Year'] == 2018]['Environmental_Growth'])
y2018=pd.Series(y2018)
from statsmodels.tsa.stattools import adfuller
X = y2018.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=2.993927, mean2=0.514992
variance1=5296.795823, variance2=4757.639775
ADF Statistic: -40.731334
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


Summary: All of the environmental growths that we used is stational.

Let's take a look at top five industries

In [ ]:
ind.groupby('Industry(Exiobase)')['Env_intensity'].count().sort_values()

Industry(Exiobase)
Cultivation of cereal grains nec                                                                                          1
Forestry, logging and related service activities (02)                                                                     5
Sea and coastal water transport                                                                                           6
Education (80)                                                                                                            6
Production of electricity by petroleum and other oil derivatives                                                         12
Mining of coal and lignite; extraction of peat (10)                                                                      15
Manufacture of tobacco products (16)                                                                                     22
Copper production                                                                                                

Top five industries:

Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)                    
Real estate activities(70)                                                                                    
Construction (45)                                                 
Manufacture of electrical machinery and apparatus n.e.c. (31)                                                 
Financial intermediation, except insurance and pension funding (65) 

In [ ]:
listind=['Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)',
'Real estate activities(70)',
'Construction (45)',
'Manufacture of electrical machinery and apparatus n.e.c. (31)',
'Financial intermediation, except insurance and pension funding (65)']
num_order_new = ind[(ind['Industry(Exiobase)']=='Construction (45)')|(ind['Industry(Exiobase)'] == 'Financial intermediation, except insurance and pension funding (65)')|(ind['Industry(Exiobase)'] == 'Manufacture of electrical machinery and apparatus n.e.c. (31)')
|(ind['Industry(Exiobase)'] == 'Real estate activities(70)')|(ind['Industry(Exiobase)'] == 'Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)')]
num_order_new  



,ISIN,Year,CompanyName,Country,Industry(Exiobase),EnvironmentalIntensity(Sales),EnvironmentalIntensity(OpInc),TotalEnvironmentalCost,WorkingCapacity,FishProductionCapacity,CropProductionCapacity,MeatProductionCapacity,Biodiversity,AbioticResources,Waterproductioncapacity(Drinkingwater&IrrigationWater),WoodProductionCapacity,SDG1.5,SDG2.1,SDG2.2,SDG2.3,SDG2.4,SDG3.3,SDG3.4,SDG3.9,SDG6,SDG12.2,SDG14.1,SDG14.2,SDG14.3,SDG14.c,SDG15.1,SDG15.2,SDG15.5,%Imputed,Env_intensity,industry_avg,industry_avg_year,Industry_indicator_year,Environmental_Growth
1,GB00B1YW4409,2010,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.12%,-0.11%,"-1,055,812","-1,032,103",-277,"-13,751","-3,221",-47,-562,"-5,953",102,"-463,300","-295,103","-294,949","-3,438","-3,438","-47,957","59,044",-74,"-5,953",-562,-4,0,-133,-4,51,51,-43,10%,-0.0012,-0.028537,-0.006402,1,NaN
2,GB00B1YW4409,2011,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.16%,-0.16%,"-961,875","-940,402",-246,"-12,525","-2,935",-42,-424,"-5,378",77,"-421,928","-264,714","-264,579","-3,131","-3,131","-42,961","44,515",-56,"-5,378",-424,-3,0,-119,-3,38,38,-39,9%,-0.0016,-0.028537,-0.009838,1,33.333333
3,GB00B1YW4409,2012,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.15%,NaN,"-722,999","-706,893",-183,"-9,414","-2,206",-32,-295,"-4,030",54,"-317,104","-197,859","-197,760","-2,354","-2,354","-32,095","30,960",-39,"-4,030",-295,-2,0,-89,-2,27,27,-30,8%,-0.0015,-0.028537,-0.024437,1,-6.250000
50,DE0005408116,2012,AAREAL BANK AG,Germany,"Financial intermediation, except insurance and...",-0.10%,-0.69%,"-1,615,657","-1,540,246",-504,"-20,570","-4,810",-79,"-1,932","-47,899",383,"-693,698","-498,714","-498,400","-5,143","-5,143","-81,852","217,411",-341,"-47,899","-1,932",-13,-2,-233,-16,192,192,-66,20%,-0.0010,-0.028537,-0.024437,1,NaN
51,DE0005408116,2013,AAREAL BANK AG,Germany,"Financial intermediation, except insurance and...",-0.11%,-0.56%,"-1,561,584","-1,483,364",-469,"-19,802","-4,632",-74,"-1,655","-51,916",328,"-667,625","-469,884","-469,596","-4,950","-4,950","-76,994","186,258",-292,"-51,916","-1,655",-11,-2,-219,-13,164,164,-63,19%,-0.0011,-0.028537,-0.025627,1,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14485,KYG989221000,2016,ZHEN DING TECHNOLOGY HOLDING LIMITED,Taiwan,Manufacture of electrical machinery and appara...,-8.73%,-155.74%,"-222,083,216","-204,825,973","-47,125","-2,713,781","-635,954","-8,532","-8,129","-13,843,967",246,"-91,295,951","-53,183,570","-53,158,871","-678,445","-678,445","-8,572,045","-630,018","-1,955","-13,843,967","-8,129",-27,-61,"-23,440",-32,123,123,"-8,504",1%,-0.0873,-0.067427,-0.073788,-1,-5.825243
14486,KYG989221000,2017,ZHEN DING TECHNOLOGY HOLDING LIMITED,Taiwan,Manufacture of electrical machinery and appara...,-6.03%,-76.11%,"-221,429,493","-203,149,289","-46,856","-2,686,271","-629,599","-8,463","-11,727","-14,897,562",273,"-90,384,665","-52,689,302","-52,665,410","-671,568","-671,568","-8,493,012","-910,285","-2,820","-14,897,562","-11,727",-36,-88,"-23,254",-43,136,136,"-8,426",1%,-0.0603,-0.067427,-0.068294,1,-30.927835
14487,KYG989221000,2018,ZHEN DING TECHNOLOGY HOLDING LIMITED,Taiwan,Manufacture of electrical machinery and appara...,-6.15%,-48.85%,"-236,938,822","-217,968,519","-51,373","-2,886,977","-676,611","-9,212","-12,305","-15,337,583","3,756","-97,158,892","-57,405,552","-57,379,583","-721,744","-721,744","-9,265,219","1,097,855","-2,959","-15,337,583","-12,305",-155,-92,"-25,366",-185,"1,878","1,878","-9,054",3%,-0.0615,-0.067427,-0.076812,1,1.990050
14488,KYG989221000,2019,ZHEN DING TECHNOLOGY HOLDING LIMITED,Taiwan,Manufacture of electrical machinery and appara...,-6.02%,-49.99%,"-241,827,228","-223,793,936","-52,466","-2,966,025","-694,995","-9,454","-12,822","-14,301,445","3,914","-99,793,897","-58,835,2

In [ ]:
y2018=list(num_order_new[num_order_new['Year'] == 2018]['Env_intensity'])
y2018=pd.Series(y2018)
from statsmodels.tsa.stattools import adfuller
X = y2018.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.041328, mean2=-0.039877
variance1=0.010222, variance2=0.005468
ADF Statistic: -21.045903
p-value: 0.000000
Critical Values:
	1%: -3.445
	5%: -2.868
	10%: -2.570


In [ ]:
y2017=list(num_order_new[num_order_new['Year'] == 2017]['Env_intensity'])
y2017=pd.Series(y2017)
from statsmodels.tsa.stattools import adfuller
X = y2017.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.041998, mean2=-0.035402
variance1=0.010698, variance2=0.002860
ADF Statistic: -5.443552
p-value: 0.000003
Critical Values:
	1%: -3.446
	5%: -2.868
	10%: -2.570


In [ ]:
y2016=list(num_order_new[num_order_new['Year'] == 2016]['Env_intensity'])
y2016=pd.Series(y2016)
from statsmodels.tsa.stattools import adfuller
X = y2016.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.044473, mean2=-0.037712
variance1=0.011978, variance2=0.002783
ADF Statistic: -20.528414
p-value: 0.000000
Critical Values:
	1%: -3.446
	5%: -2.868
	10%: -2.570


In [ ]:
y2015=list(num_order_new[num_order_new['Year'] == 2015]['Env_intensity'])
y2015=pd.Series(y2015)
from statsmodels.tsa.stattools import adfuller
X = y2015.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.043269, mean2=-0.028244
variance1=0.012533, variance2=0.019730
ADF Statistic: -18.508807
p-value: 0.000000
Critical Values:
	1%: -3.449
	5%: -2.870
	10%: -2.571


Let's check the top 3 industries

In [ ]:
num_order_new = ind[(ind['Industry(Exiobase)']=='Construction (45)')|(ind['Industry(Exiobase)'] == 'Financial intermediation, except insurance and pension funding (65)')|(ind['Industry(Exiobase)'] == 'Manufacture of electrical machinery and apparatus n.e.c. (31)')]

In [ ]:
y2018=list(num_order_new[num_order_new['Year'] == 2018]['Env_intensity'])
y2018=pd.Series(y2018)
from statsmodels.tsa.stattools import adfuller
X = y2018.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.045647, mean2=-0.045448
variance1=0.012042, variance2=0.006497
ADF Statistic: -19.134100
p-value: 0.000000
Critical Values:
	1%: -3.449
	5%: -2.870
	10%: -2.571


In [ ]:
y2017=list(num_order_new[num_order_new['Year'] == 2017]['Env_intensity'])
y2017=pd.Series(y2017)
from statsmodels.tsa.stattools import adfuller
X = y2017.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.039995, mean2=-0.040480
variance1=0.003754, variance2=0.003352
ADF Statistic: -19.354295
p-value: 0.000000
Critical Values:
	1%: -3.448
	5%: -2.870
	10%: -2.571


In [ ]:
y2016=list(num_order_new[num_order_new['Year'] == 2016]['Env_intensity'])
y2016=pd.Series(y2016)
from statsmodels.tsa.stattools import adfuller
X = y2016.values
result = adfuller(X)
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

mean1=-0.048769, mean2=-0.043516
variance1=0.014048, variance2=0.003285
ADF Statistic: -19.089448
p-value: 0.000000
Critical Values:
	1%: -3.449
	5%: -2.870
	10%: -2.571


## Conclusion

Summary: All of the data that we used is stational.



Next, we will continue our analysis in the 'DistilBERT_CompaniesDescription' notebook